In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam

In [2]:
data = pd.read_csv("Bank_Personal_Loan_Modelling.csv")

In [3]:
data

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [5]:
data.describe()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,93152.503000,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,2121.852197,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,9307.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,91911.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,93437.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,94608.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,96651.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [6]:
col = data.columns
col =[ 'Age', 'Experience', 'Income', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Securities Account',
       'CD Account', 'Online', 'CreditCard','Personal Loan']
data = data[col]

In [7]:
data.duplicated().sum()

13

In [8]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values
x.shape,y.shape

((5000, 11), (5000,))

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=69)
(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

((4000, 11), (1000, 11), (4000,), (1000,))

In [10]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# x_train = sc.fit_transform(x_train)

In [11]:
X_train = torch.from_numpy(x_train).to(torch.float32)
Y_train = torch.from_numpy(y_train)

### DATALOADER ITERATOR

In [12]:
df = TensorDataset(X_train,Y_train)
data_load = DataLoader(df,batch_size=5)

### Multilayer perceptron model

In [13]:
class NeuralNet(nn.Module):
    def __init__(self,x,h,y):
        super().__init__()
        self.l1 = nn.Linear(x,h)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(h,y)
    def forward(self,val):
        val = self.l1(val)
        val = self.relu(val)
        val = self.l2(val)
        return val    

In [14]:
#input_size
x = X_train.shape[1]
# hidden layer size
h = X_train.shape[1] #most preferably the size of input
#output size
y = 2
mod = NeuralNet(x,h,y)
x,y,h

(11, 2, 11)

In [15]:
x = data_load.dataset

### Loss and optimizing

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(mod.parameters(),lr=0.003)


In [17]:
epoch = 50
pr=0
training_loss = [0] * epoch
training_accuracy = [0] * epoch
for i in range (epoch):
    
    for x1,y1 in data_load:
        #forward
        output = mod(x1)
        loss = criterion(output,y1)
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        training_loss[i] += loss.item() * y1.size(0)
        correct_count = (torch.argmax(output, dim=1) == y1).to(torch.float32)
        training_accuracy[i] += correct_count.mean() * y1.size(0)
    training_loss[i] /= len(data_load.dataset)
    training_accuracy[i] /= len(data_load.dataset)
    
    pr = loss.item()    
    print("Epoch:",i+1,"Error:",training_loss[i],training_accuracy[i])    

Epoch: 1 Error: 0.45946659993038336 tensor(0.8670)
Epoch: 2 Error: 0.26593798677133235 tensor(0.8963)
Epoch: 3 Error: 0.21843961711120755 tensor(0.9105)
Epoch: 4 Error: 0.19208239620228595 tensor(0.9247)
Epoch: 5 Error: 0.17361374467701352 tensor(0.9327)
Epoch: 6 Error: 0.16022509852150846 tensor(0.9335)
Epoch: 7 Error: 0.15169772936761972 tensor(0.9385)
Epoch: 8 Error: 0.1465478278647271 tensor(0.9415)
Epoch: 9 Error: 0.1379802629581019 tensor(0.9448)
Epoch: 10 Error: 0.13063329630660467 tensor(0.9470)
Epoch: 11 Error: 0.12513998253174305 tensor(0.9525)
Epoch: 12 Error: 0.1212719583032337 tensor(0.9542)
Epoch: 13 Error: 0.11438294236028199 tensor(0.9565)
Epoch: 14 Error: 0.11070373757784267 tensor(0.9590)
Epoch: 15 Error: 0.10680468349009914 tensor(0.9590)
Epoch: 16 Error: 0.10436247136636212 tensor(0.9600)
Epoch: 17 Error: 0.10247139434672604 tensor(0.9605)
Epoch: 18 Error: 0.10400227692307454 tensor(0.9607)
Epoch: 19 Error: 0.10367380759632702 tensor(0.9613)
Epoch: 20 Error: 0.09995

In [163]:
import random

In [196]:
# Accuracy 
def fitness_function(model):
    
    output=model(X_train)
   
    correct_count = (torch.argmax(output, dim=1) == Y_train).to(torch.float32)
    #print(correct_count.sum())
    training_accuracy = correct_count.sum().item()/len(data_load.dataset)
    return training_accuracy

In [197]:
def mutation(mat,child):
    #mutation
    #making the mutation with the random ranges genarated
    s = random.randrange(0,len(mat)//2)
    e = random.randrange(s,len(mat))
    mut_child = child.copy()
    mut_child[s:e] = mut_child[s:e][::-1] 
    return mut_child

In [203]:
def flattening(model):
    matrix=[]
    for parameters in model.parameters():
        #print(parameters)
        matrix.append(parameters.numpy().flatten())
         
    return np.concatenate(matrix) 

In [204]:
def reshape(child,shape):
    size=[]
    for i in shape:
        if(len(i)==2):
            size+=[i[0]*i[1]]
        else:
            size+=[i[0]]
    param = []
    offset = 0
    for i in range(len(shape)):
        parameter = child[offset : offset + size[i]].reshape(shape[i])
        param.append(parameter)
        offset+=size[i]
    return param

In [205]:
# Crossover and Mutation
def crossover(model1,model2):
    shape = [i.numpy().shape for i in model1.parameters()]
    
        
    #flattening
    mat_1 = flattening(model1)
    mat_2 = flattening(model2)
    
    #crossover
    ##choosing the crossover point
    start = len(mat_1)//2 - 10
    end = len(mat_1)//2 + 10
    mid = random.randrange(start,end)
    
    ##crossover process
    child_1 = np.concatenate([mat_1[:mid],mat_2[mid:]])
    child_2 = np.concatenate([mat_2[:mid],mat_1[mid:]])
    
    ##random mutation point
    
    m_child1 = mutation(mat_1,child_1)
    m_child2 = mutation(mat_1,child_2)
    
    #print(m_child1)
    children = [child_1,child_2,m_child1,m_child2]
    output=[]
    
    for child in children:
        output.append(reshape(child,shape))
    output = np.array(output, dtype="object")
    return output

In [206]:
# Training 

torch.manual_seed(420)
torch.set_grad_enabled(False)
population_size = 10

def train(no):
    # Initial Population
    population = np.array([NeuralNet(11,11,2) for i in range(population_size)])
    #print(population)
    best_model = None
    
    for loop in range(no):
        # Sort population based up on fitness function
        population = population[np.argsort([fitness_function(model) for model in population])]
        
        # Printing Max Accuracy
        best_model = population[-1]
        print("Gen", loop+1, " :", fitness_function(population[-1]))
        parent1 = population[-1]
        parent2 = population[-2]
        parent3 = population[-3]
        parent4 = population[-4] 
        parent_1 = population[0]
        parent_2 = population[1]
        
        # Crossover
        output_1 = crossover(parent1, parent3)
        output_2 = crossover(parent2, parent4)
        output = np.concatenate([output_1, output_2])
        
        # New population
        new_population = np.array([NeuralNet(11,11,2) for i in range(len(output))])
        for count, model in enumerate(new_population, 0):
            for index, param in enumerate(model.parameters(), 0):
                param.data = (torch.tensor(output[count][index]))
        
        # Add Poor performing parent to the new population
        new_population = np.concatenate([new_population, [parent_1, parent_2]])
        
        # New population becomes your next population
        population = new_population.copy()
        
    return best_model

In [207]:
best_model = train(1000)

Gen 1  : 0.903
Gen 2  : 0.903
Gen 3  : 0.903
Gen 4  : 0.903
Gen 5  : 0.903
Gen 6  : 0.903
Gen 7  : 0.903
Gen 8  : 0.903
Gen 9  : 0.903
Gen 10  : 0.903
Gen 11  : 0.903
Gen 12  : 0.903
Gen 13  : 0.903
Gen 14  : 0.903
Gen 15  : 0.903
Gen 16  : 0.903
Gen 17  : 0.903
Gen 18  : 0.903
Gen 19  : 0.903
Gen 20  : 0.903
Gen 21  : 0.903
Gen 22  : 0.903
Gen 23  : 0.903
Gen 24  : 0.903
Gen 25  : 0.903
Gen 26  : 0.903
Gen 27  : 0.903
Gen 28  : 0.903
Gen 29  : 0.903
Gen 30  : 0.903
Gen 31  : 0.903
Gen 32  : 0.903
Gen 33  : 0.903
Gen 34  : 0.903
Gen 35  : 0.903
Gen 36  : 0.903
Gen 37  : 0.903
Gen 38  : 0.903
Gen 39  : 0.903
Gen 40  : 0.903
Gen 41  : 0.903
Gen 42  : 0.903
Gen 43  : 0.903
Gen 44  : 0.903
Gen 45  : 0.903
Gen 46  : 0.903
Gen 47  : 0.903
Gen 48  : 0.903
Gen 49  : 0.903
Gen 50  : 0.903
Gen 51  : 0.903
Gen 52  : 0.903
Gen 53  : 0.903
Gen 54  : 0.903
Gen 55  : 0.903
Gen 56  : 0.903
Gen 57  : 0.903
Gen 58  : 0.903
Gen 59  : 0.903
Gen 60  : 0.903
Gen 61  : 0.903
Gen 62  : 0.903
Gen 63  : 0.903
G

Gen 490  : 0.903
Gen 491  : 0.903
Gen 492  : 0.903
Gen 493  : 0.903
Gen 494  : 0.903
Gen 495  : 0.903
Gen 496  : 0.903
Gen 497  : 0.903
Gen 498  : 0.903
Gen 499  : 0.903
Gen 500  : 0.903
Gen 501  : 0.903
Gen 502  : 0.903
Gen 503  : 0.903
Gen 504  : 0.903
Gen 505  : 0.903
Gen 506  : 0.903
Gen 507  : 0.903
Gen 508  : 0.903
Gen 509  : 0.903
Gen 510  : 0.903
Gen 511  : 0.903
Gen 512  : 0.903
Gen 513  : 0.903
Gen 514  : 0.903
Gen 515  : 0.903
Gen 516  : 0.903
Gen 517  : 0.903
Gen 518  : 0.903
Gen 519  : 0.903
Gen 520  : 0.903
Gen 521  : 0.903
Gen 522  : 0.903
Gen 523  : 0.903
Gen 524  : 0.903
Gen 525  : 0.903
Gen 526  : 0.903
Gen 527  : 0.903
Gen 528  : 0.903
Gen 529  : 0.903
Gen 530  : 0.903
Gen 531  : 0.903
Gen 532  : 0.903
Gen 533  : 0.903
Gen 534  : 0.903
Gen 535  : 0.903
Gen 536  : 0.903
Gen 537  : 0.903
Gen 538  : 0.903
Gen 539  : 0.903
Gen 540  : 0.903
Gen 541  : 0.903
Gen 542  : 0.903
Gen 543  : 0.903
Gen 544  : 0.903
Gen 545  : 0.903
Gen 546  : 0.903
Gen 547  : 0.903
Gen 548  : 0.9

Gen 972  : 0.903
Gen 973  : 0.903
Gen 974  : 0.903
Gen 975  : 0.903
Gen 976  : 0.903
Gen 977  : 0.903
Gen 978  : 0.903
Gen 979  : 0.903
Gen 980  : 0.903
Gen 981  : 0.903
Gen 982  : 0.903
Gen 983  : 0.903
Gen 984  : 0.903
Gen 985  : 0.903
Gen 986  : 0.903
Gen 987  : 0.903
Gen 988  : 0.903
Gen 989  : 0.903
Gen 990  : 0.903
Gen 991  : 0.903
Gen 992  : 0.903
Gen 993  : 0.903
Gen 994  : 0.903
Gen 995  : 0.903
Gen 996  : 0.903
Gen 997  : 0.903
Gen 998  : 0.903
Gen 999  : 0.903
Gen 1000  : 0.903


[(11, 11), (11,), (2, 11), (2,)]
